<a href="https://colab.research.google.com/github/senecamiller/Coding_02_Final_Project/blob/main/Cleaning_C2_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!!! DATA CLEANING !!!

In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving final_df.csv to final_df.csv


In [ ]:
import pandas as pd

# Load the uploaded file
final_df = pd.read_csv('final_df.csv')
print(final_df.head())

         price  Location  Year    TTAF Serial number            Registration  \
0  COMING SOON    Poland  2021    490h       24-0345    SP-WKX (EASA-Poland)   
1    460 000 $    Poland  1974  15877h     172S10770    SP-THL (EASA-Poland)   
2  COMING SOON    Poland  1978   4654h     F15001127    SP-KIZ (EASA-Poland)   
3  COMING SOON  Portugal  1959   2830h     T20608855  SP-HAV (EASA - Poland)   
4  419 000 â¬    Poland  1991   3392h          3630  SP-SZP (EASA - Poland)   

                       Reason of sale  \
0  Other aviation project in progress   
1  Other aviation project in progress   
2  Other aviation project in progress   
3  Other aviation project in progress   
4  Other aviation project in progress   

                       Owned by private person Availability  Type of flights  \
0  Owned by company - selling based on invoice  Immediately  Private Flights   
1  Owned by company - selling based on invoice  Immediately  Private Flights   
2                               

In [ ]:
# Keep the original DataFrame intact and create a cleaned version (copy)
final_df_copy = final_df.copy()

# Make Column Names Lowercase (easier to explore)
final_df_copy.columns = final_df_copy.columns.str.lower()

# Drop all columns that contain more than 100 null variables
final_df_cleaned = final_df_copy.loc[:, final_df_copy.isna().sum() <= 100]

In [ ]:
# Used final_df_cleaned['column-name'].unique() and final_df_cleaned['column-name'].nunique() to learn about the data
# For more info I used some_list = list(final_df_cleaned['column-name']) then some_list.count('value-I-wanted-counted')

# In 'price' category, change USD -> $ (for consistency, both are present) and â\x82¬ -> € (website confirms the two are equal)
final_df_cleaned.loc[:, 'price'] = final_df_cleaned['price'].replace(r'[^\d.,]', '', regex=True)
final_df_cleaned.loc[:, 'price'] = final_df_cleaned['price'].str.replace('USD', '$', regex=False).str.replace('â\x82¬', '€', regex=False)
final_df_cleaned.loc[:, 'price'] = pd.to_numeric(final_df_cleaned['price'], errors='coerce')

# Convert similar column values in 'taxes' to same
# many variations of "Price + VAT (VAT rate depends on Buyer's status)" -- ex: case insensitive
final_df_cleaned.loc[:, 'taxes'] = final_df_cleaned['taxes'].apply(
    lambda x: 'Price + VAT (VAT rate depends on Buyer\'s status)'
    if isinstance(x, str) and 'price + vat' in x.lower() and 'vat rate' in x.lower() else x
)

In [ ]:
# Clean 'visual condition' column to remove duplicate values (8//10 vs 8/10 and 9.5/10 vs 9,5/10) and remove /10 to make visualizations easier
final_df_cleaned.loc[:, 'visual condition'] = final_df_cleaned['visual condition'].apply(
    lambda x: '10' if isinstance(x, str) and '10/10' in x
    else '9' if isinstance(x, str) and '9/10' in x
    else '9.5' if isinstance(x, str) and '9,5' in x
    else '8' if isinstance(x, str) and '8//10' in x
    else x.replace('/10', '') if isinstance (x,str) and '/10' in x
    else x
)

# Clean 'owner' into private owner vs. a company selling the plane
final_df_cleaned.loc[:, 'owner'] = final_df_cleaned['owner'].apply(
    lambda x: 'Private Owner' if isinstance(x, str) and ('owner' in x.lower() or 'person' in x.lower() or 'vat paid' in x.lower())
    else 'Private Company' if isinstance(x, str) and ('company' in x.lower() or 'local aeroclub' in x.lower())
    else x
)

# Remove row with year = '.' and ttaf = '.' (1 row each)
final_df_cleaned = final_df_cleaned[final_df_cleaned['price'] != '.']
final_df_cleaned = final_df_cleaned[final_df_cleaned['ttaf'] != '.']

In [ ]:
final_df_cleaned.to_csv('final_df_cleaned.csv',   index=False)

In [ ]:
from google.colab import files
files.download('final_df_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>